In [1]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

debug=False

In [2]:
# Mosaic multiple files
mosaic_ts_info = False
from rasterio.merge import merge

# ====== MERGE CALIBRATED GLACIERS INTO DEGREE PIXELS ======
# Export all .tifs to a common projection that will enable merging them smoothly
dst_crs = 'EPSG:4326'

if mosaic_ts_info:
    
    # All Ts filenames
    ts_fullfns = []
    ts_year_fullfns = []
    ts_doy_fullfns = []
    ts_dayfrac_fullfns = []
    for i in os.listdir(debris_prms.ts_fp):
        if i.endswith('.tif'):
            if 'tsurfC' in i:
                ts_fullfns.append(debris_prms.ts_fp + i)
            elif 'year' in i:
                ts_year_fullfns.append(debris_prms.ts_fp + i)
            elif 'doy' in i:
                ts_doy_fullfns.append(debris_prms.ts_fp + i)
            elif 'dayfrac' in i:
                ts_dayfrac_fullfns.append(debris_prms.ts_fp + i)

    ts_fullfns = sorted(ts_fullfns)
    ts_year_fullfns = sorted(ts_year_fullfns)
    ts_doy_fullfns = sorted(ts_doy_fullfns)
    ts_dayfrac_fullfns = sorted(ts_dayfrac_fullfns)
    
    if debug:
        print(ts_fullfns)
        print(ts_year_fullfns)
        print(ts_doy_fullfns)
        print(ts_dayfrac_fullfns)

    def mosaic_fullfns(list_fullfns, fn_merged=None):
        if fn_merged is None:
            fn_merged = list_fullfns[0].split('/')[-1].split('-')[0] + '.tif'
        src_files_to_mosaic = []
        for fn in list_fullfns:
            with rasterio.open(fn) as src:
                src = rasterio.open(fn)
                src_files_to_mosaic.append(src)

        mos, out_trans = merge(src_files_to_mosaic)

        with rasterio.open(debris_prms.ts_fp + fn_merged,"w", driver ='Gtiff',count=1,
                           height= mos.shape[1],
                           width= mos.shape[2],
                           transform= out_trans,
                           crs= src.crs,
                           dtype= src.dtypes[0]) as dest:
            dest.write(mos)
            
    mosaic_fullfns(ts_fullfns)
    mosaic_fullfns(ts_year_fullfns)
    mosaic_fullfns(ts_doy_fullfns)
    mosaic_fullfns(ts_dayfrac_fullfns)

In [3]:
ts_fullfns = debris_prms.ts_fullfns_dict[debris_prms.roi]
ts_year_fullfns = debris_prms.ts_year_fullfns_dict[debris_prms.roi]
ts_doy_fullfns = debris_prms.ts_doy_fullfns_dict[debris_prms.roi]
ts_dayfrac_fullfns = debris_prms.ts_dayfrac_fullfns_dict[debris_prms.roi]

In [ ]:
add_ts_fns = True
if add_ts_fns:
    
    # Load all debris-covered glaciers
    dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
    dc_shp = dc_shp.sort_values(by=['RGIId'])
    dc_shp.reset_index(inplace=True, drop=True)
    
    dc_rgiids = [x.split('-')[1] for x in dc_shp.RGIId.values]
    print(len(dc_rgiids), 'ex:', dc_rgiids[0],'\n')
    main_glac_rgi_all = debris_prms.selectglaciersrgitable(dc_rgiids)

    # Ts filenames dataframe
    if os.path.exists(debris_prms.ts_fp + debris_prms.ts_fns_fn):
        ts_fns_df = pd.read_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn)
    else:
        ts_fns_cns = ['RGIId', 'ts_fullfn', 'ts_year_fullfn', 'ts_doy_fullfn', 'ts_dayfrac_fullfn']
        ts_fns_df = pd.DataFrame(np.zeros((main_glac_rgi_all.shape[0], len(ts_fns_cns))), columns=ts_fns_cns)
        ts_fns_df['RGIId'] = main_glac_rgi_all['RGIId']
        
    if debug:
        print(ts_fullfns)
        print(ts_year_fullfns)
        print(ts_doy_fullfns)
        print(ts_dayfrac_fullfns)
        
    ts_fn_wglacier = None
    ts_year_fn_wglacier = None
    ts_doy_fn_wglacier = None
    ts_dayfrac_fn_wglacier = None
    
    # ===== PROCESS EACH GLACIER =====
    for nglac, glac_idx in enumerate(main_glac_rgi_all.index.values):
#     for nglac, glac_idx in enumerate([main_glac_rgi_all.index.values[0]]):
        glac_str = main_glac_rgi_all.loc[glac_idx,'rgino_str']
        rgiid = main_glac_rgi_all.loc[glac_idx,'RGIId']
        region = glac_str.split('.')[0]

        if int(region) < 10:
            glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
        else:
            glac_str_noleadzero = glac_str

        print(nglac, glac_idx, rgiid)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'

        if os.path.exists(thick_dir + thick_fn):
#         if os.path.exists(thick_dir + thick_fn) and '0.0' in ts_fns_df.loc[glac_idx,:].values:
#             print(nglac, glac_idx, rgiid)

            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()

            # ==== CHECK IF TIF HAS SURFACE TEMPERATURE DATA OVER THE GLACIER =====
            # Add the filenames
            fn_dict = OrderedDict()
            # DEM
            z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
            z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
            fn_dict['z1'] = z1_fp + z1_fn
            # Ice thickness
            thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
            thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
            fn_dict['ice_thick'] = thick_dir + thick_fn
            # Ts
            ts_dict_keys = []
            for nfn, ts_fn in enumerate(ts_fullfns):
                fn_dict['ts_' + str(nfn)] = ts_fn
                ts_dict_keys.append('ts_' + str(nfn))
            ts_year_dict_keys = []
            for nfn, ts_year_fn in enumerate(ts_year_fullfns):
                fn_dict['ts_year_' + str(nfn)] = ts_year_fn
                ts_year_dict_keys.append('ts_year_' + str(nfn))
            ts_doy_dict_keys = []
            for nfn, ts_doy_fn in enumerate(ts_doy_fullfns):
                fn_dict['ts_doy_' + str(nfn)] = ts_doy_fn
                ts_doy_dict_keys.append('ts_doy_' + str(nfn))
            ts_dayfrac_dict_keys = []  
            for nfn, ts_dayfrac_fn in enumerate(ts_dayfrac_fullfns):
                fn_dict['ts_dayfrac_' + str(nfn)] = ts_dayfrac_fn
                ts_dayfrac_dict_keys.append('ts_dayfrac_' + str(nfn))

            # ===== PROCESS THE DATA =====
            #Expand extent to include buffered region around glacier polygon
            warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
            #Warp everything to common res/extent/proj
            z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
            z1_res = np.min([z1_gt[1], -z1_gt[5]])
            ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                               t_srs=gf.aea_srs, verbose=False, r='cubic')
            ds_dict = dict(zip(fn_dict.keys(), ds_list))
            gf.ds_dict = ds_dict

            if 'z1' in ds_dict:
                #This is False over glacier polygon surface, True elsewhere - can be applied directly
                glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                glac_geom_mask_copy = glac_geom_mask.copy()
                gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                # Debris cover
                dc_shp_lyr_mask = geolib.lyr2mask(dc_shp_lyr, ds_dict['ice_thick'])
                gf.dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
                gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=gf.dc_mask)

                # ====== SURFACE TEMPERATURE =====
                find_ts = True
                ts_key_count = 0
                if debug:
                    print('\nSURFACE TEMPERATURE')
                while find_ts and ts_key_count < len(ts_dict_keys):
                    ts_key = ts_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts = False
                        # Export raster
                        ts_fn = rgiid + '_ts.tif'
                        ts_fp = debris_prms.oggm_ts_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_fp):
                            os.makedirs(ts_fp)
                        iolib.writeGTiff(gf.ts, ts_fp + ts_fn, gf.ds_dict['z1'])
                        ts_fn_wglacier = ts_fp + ts_fn
                        
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_fn:', ts_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = malib.calcperc(var_full2plot, (2,98))
                            plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'Ts (degC)', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS YEAR =====
                find_ts_year = True
                ts_key_count = 0
                if debug:
                    print('\nYEAR')
                while find_ts_year and ts_key_count < len(ts_year_dict_keys):
                    ts_key = ts_year_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_year_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts_year = False
                        # Export raster
                        ts_year_fn = rgiid + '_ts_year.tif'
                        ts_year_fp = debris_prms.oggm_ts_info_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_year_fp):
                            os.makedirs(ts_year_fp)
                        iolib.writeGTiff(gf.ts, ts_year_fp + ts_year_fn, gf.ds_dict['z1'])
                        ts_year_fn_wglacier = ts_year_fp + ts_year_fn
                        
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_year_fn:', ts_year_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (2013,2018)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_year'], 'inferno', 'Year', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS DOY =====
                find_ts_doy = True
                ts_key_count = 0
                if debug:
                    print('\nDAY OF YEAR')
                while find_ts_doy and ts_key_count < len(ts_doy_dict_keys):
                    ts_key = ts_doy_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_doy_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts_doy = False
                        # Export raster
                        ts_doy_fn = rgiid + '_ts_doy.tif'
                        ts_doy_fp = debris_prms.oggm_ts_info_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_doy_fp):
                            os.makedirs(ts_doy_fp)
                        iolib.writeGTiff(gf.ts, ts_doy_fp + ts_doy_fn, gf.ds_dict['z1'])
                        ts_doy_fn_wglacier = ts_doy_fp + ts_doy_fn
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_doy_fn:', ts_doy_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (160,280)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_doy'], 'inferno', 'DOY', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                # ====== TS DAYFRAC =====
                find_ts_dayfrac = True
                ts_key_count = 0
                if debug:
                    print('\nDAY FRACTION')
                while find_ts_dayfrac and ts_key_count < len(ts_dayfrac_dict_keys):
                    ts_key = ts_dayfrac_dict_keys[ts_key_count]
                    if debug:
                        print(ts_key, ts_dayfrac_fullfns[ts_key_count].split('/')[-1])
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.ts.mask = np.ma.mask_or(
                        glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.ts.data, np.isnan(gf.ts.data))))
                    gf.dc_ts = np.ma.array(iolib.ds_getma(ds_dict[ts_key]), mask=glac_geom_mask_copy)
                    gf.dc_ts.mask = gf.dc_mask
                    # Count Ts pixels
                    dc_ts_pixels = len(gf.ts.nonzero()[0])
                    if dc_ts_pixels / gf.dc_area.count() * 100 > 75:
                        find_ts_dayfrac = False
                        # Export raster
                        ts_dayfrac_fn = rgiid + '_ts_dayfrac.tif'
                        ts_dayfrac_fp = debris_prms.oggm_ts_info_fp + 'RGI60-' + debris_prms.roi + '/'
                        if not os.path.exists(ts_dayfrac_fp):
                            os.makedirs(ts_dayfrac_fp)
                        iolib.writeGTiff(gf.ts, ts_dayfrac_fp + ts_dayfrac_fn, gf.ds_dict['z1'])
                        ts_dayfrac_fn_wglacier = ts_dayfrac_fp + ts_dayfrac_fn
                        if debug:
                            print('\n# z1 dc pixels:', gf.dc_area.count())
                            print('# dc_ts_pixels:', dc_ts_pixels)
                            print('ts_dayfrac_fn:', ts_dayfrac_fn_wglacier)
                            var_full2plot = gf.ts.copy()
                            clim = (0,24)
                            plot_array(var_full2plot, clim, [glac_str + ' Ts_dayfrac'], 'inferno', 'dayfrac', 
                                       close_fig=False)
                    ts_key_count += 1
                    
                
                        
            # ===== Add layers =====
            # Record filenames if all exist
            if (ts_fn_wglacier is not None and ts_year_fn_wglacier is not None and ts_doy_fn_wglacier is not None and 
                ts_dayfrac_fn_wglacier is not None):                    
                ts_fns_df.loc[glac_idx,:] = [rgiid, ts_fn_wglacier, ts_year_fn_wglacier, ts_doy_fn_wglacier, 
                                             ts_dayfrac_fn_wglacier]
                    
    # Save updated filenames
    ts_fns_df.to_csv(debris_prms.ts_fp + debris_prms.ts_fns_fn, index=False)

1011 ex: 18.00002 

1011 glaciers in region 18 are included in this model run: ['00002', '00004', '00009', '00011', '00013', '00019', '00020', '00021', '00024', '00025', '00028', '00029', '00030', '00031', '00032', '00034', '00035', '00036', '00037', '00038', '00040', '00041', '00042', '00044', '00045', '00046', '00047', '00048', '00049', '00050', '00051', '00052', '00054', '00055', '00056', '00057', '00058', '00059', '00062', '00063', '00065', '00066', '00067', '00069', '00070', '00071', '00072', '00073', '00074', '00076'] and more
This study is focusing on 1011 glaciers in region [18]
0 0 RGI60-18.00002
1 1 RGI60-18.00004
2 2 RGI60-18.00009
3 3 RGI60-18.00011
4 4 RGI60-18.00013
5 5 RGI60-18.00019
6 6 RGI60-18.00020
7 7 RGI60-18.00021
8 8 RGI60-18.00024
9 9 RGI60-18.00025
10 10 RGI60-18.00028
11 11 RGI60-18.00029
12 12 RGI60-18.00030
13 13 RGI60-18.00031
14 14 RGI60-18.00032
15 15 RGI60-18.00034
16 16 RGI60-18.00035
17 17 RGI60-18.00036
18 18 RGI60-18.00037
19 19 RGI60-18.00038
20 20 

341 341 RGI60-18.00558
342 342 RGI60-18.00561
343 343 RGI60-18.00563
344 344 RGI60-18.00565
345 345 RGI60-18.00566
346 346 RGI60-18.00568
347 347 RGI60-18.00569
348 348 RGI60-18.00570
349 349 RGI60-18.00572
350 350 RGI60-18.00575
351 351 RGI60-18.00576
352 352 RGI60-18.00577
353 353 RGI60-18.00578
354 354 RGI60-18.00579
355 355 RGI60-18.00580
356 356 RGI60-18.00581
357 357 RGI60-18.00588
358 358 RGI60-18.00589
359 359 RGI60-18.00591
360 360 RGI60-18.00592
361 361 RGI60-18.00593
362 362 RGI60-18.00595
363 363 RGI60-18.00599
364 364 RGI60-18.00601
365 365 RGI60-18.00602
366 366 RGI60-18.00603
367 367 RGI60-18.00604
368 368 RGI60-18.00605
369 369 RGI60-18.00607
370 370 RGI60-18.00609
371 371 RGI60-18.00611
372 372 RGI60-18.00612
373 373 RGI60-18.00613
374 374 RGI60-18.00615
375 375 RGI60-18.00617
376 376 RGI60-18.00623
377 377 RGI60-18.00625
378 378 RGI60-18.00626
379 379 RGI60-18.00629
380 380 RGI60-18.00631
381 381 RGI60-18.00633
382 382 RGI60-18.00636
383 383 RGI60-18.00637
384 384 RGI

698 698 RGI60-18.01871
699 699 RGI60-18.01875
700 700 RGI60-18.01881
701 701 RGI60-18.01882
702 702 RGI60-18.01889
703 703 RGI60-18.01894
704 704 RGI60-18.01895
705 705 RGI60-18.01903
706 706 RGI60-18.01908
707 707 RGI60-18.01917
708 708 RGI60-18.01934
709 709 RGI60-18.01935
710 710 RGI60-18.01936
711 711 RGI60-18.01939
712 712 RGI60-18.01951
713 713 RGI60-18.01956
714 714 RGI60-18.01958
715 715 RGI60-18.01959
716 716 RGI60-18.01960
717 717 RGI60-18.01961
718 718 RGI60-18.01968
719 719 RGI60-18.01974
720 720 RGI60-18.01976
721 721 RGI60-18.01978
722 722 RGI60-18.01986
723 723 RGI60-18.02010
724 724 RGI60-18.02014
725 725 RGI60-18.02018
726 726 RGI60-18.02040
727 727 RGI60-18.02045
728 728 RGI60-18.02051
729 729 RGI60-18.02056
730 730 RGI60-18.02060
731 731 RGI60-18.02061
732 732 RGI60-18.02062
733 733 RGI60-18.02063
734 734 RGI60-18.02069
735 735 RGI60-18.02092
736 736 RGI60-18.02093
737 737 RGI60-18.02094
738 738 RGI60-18.02100
739 739 RGI60-18.02142
740 740 RGI60-18.02147
741 741 RGI